In [4]:
!pip show json

In [6]:
import os
import json
import numpy as np
import cv2

def json_to_mask(json_path, img_shape, save_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    mask = np.zeros(img_shape, dtype=np.uint8)

    for obj in data.get('shapes', []):  # Ensure 'shapes' key exists
        points = np.array(obj['points'], dtype=np.int32)
        cv2.fillPoly(mask, [points], color=255)

    cv2.imwrite(save_path, mask)
    print(f"✅ Mask saved at: {save_path}")

# ✅ Update paths
json_file = r"C:/Users/Het/Desktop/Het/Method-2/images/GDL.json"  # Single JSON file
img_folder = r"C:/Users/Het/Desktop/Het/Method-2/images/GDL"  # Folder with images
mask_folder = r"C:/Users/Het/Desktop/Het/Method-2/images/GDL_mask"  # Folder for masks

os.makedirs(mask_folder, exist_ok=True)  # Ensure mask folder exists

# Extract image name from JSON filename
img_name = os.path.splitext(os.path.basename(json_file))[0] + ".png"  # Change extension if needed
img_path = os.path.join(img_folder, img_name)
mask_path = os.path.join(mask_folder, img_name)  # Save mask with same name as image

if os.path.exists(json_file) and os.path.exists(img_path):
    img_shape = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).shape  # Get image dimensions
    json_to_mask(json_file, img_shape, mask_path)
else:
    print("❌ Error: JSON file or corresponding image file not found!")

❌ Error: JSON file or corresponding image file not found!


In [7]:
import os

json_file = r"C:/Users/Het/Desktop/Het/Method-2/images/GDL.json"

if os.path.exists(json_file):
    print("✅ JSON file exists!")
else:
    print("❌ JSON file NOT found. Check the file path.")

✅ JSON file exists!


In [9]:
import os

img_folder = r"C:/Users/Het/Desktop/Het/Method-2/images/GDL"  # Image folder
img_name = "GDL.JPG"  # Change if it's JPG, TIFF, etc.
img_path = os.path.join(img_folder, img_name)

if os.path.exists(img_path):
    print(f"✅ Image file '{img_name}' exists in {img_folder}!")
else:
    print(f"❌ Image file '{img_name}' NOT found in {img_folder}. Check the file extension.")

❌ Image file 'GDL.JPG' NOT found in C:/Users/Het/Desktop/Het/Method-2/images/GDL. Check the file extension.


In [12]:
import os
import json
import numpy as np
import cv2

def json_to_mask(json_path, img_shape, save_path):
    """Convert JSON annotation to a mask."""
    with open(json_path, 'r') as f:
        data = json.load(f)

    mask = np.zeros(img_shape, dtype=np.uint8)

    for obj in data.get('shapes', []):  # Ensure 'shapes' key exists
        points = np.array(obj['points'], dtype=np.int32)
        cv2.fillPoly(mask, [points], color=255)

    cv2.imwrite(save_path, mask)
    print(f"✅ Mask saved at: {save_path}")

# ✅ Update paths
json_folder = r"C:/Users/Het/Desktop/Het/Method-2/images/"  # JSON file location
img_folder = r"C:/Users/Het/Desktop/Het/Method-2/images/IMP/"  # Folder with images
mask_folder = r"C:/Users/Het/Desktop/Het/Method-2/images/GDL_mask/"  # Folder for masks

os.makedirs(mask_folder, exist_ok=True)  # Ensure mask folder exists

# ✅ Find the JSON file
json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]
if not json_files:
    print("❌ No JSON files found!")
    exit()

for json_file in json_files:
    json_path = os.path.join(json_folder, json_file)
    base_name = os.path.splitext(json_file)[0]  # Remove .json extension

    # ✅ Find the corresponding image (starts with "GDL-")
    img_candidates = [f for f in os.listdir(img_folder) if f.startswith("GDL-")]
    
    if not img_candidates:
        print(f"❌ No image files found for {json_file}")
        continue

    # ✅ Assume the first matching image is the right one
    img_name = img_candidates[0]  # Take the first matching image
    img_path = os.path.join(img_folder, img_name)

    if os.path.exists(img_path):
        img_shape = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).shape  # Get image dimensions
        mask_path = os.path.join(mask_folder, base_name + "_mask.png")  # Save mask with modified name
        json_to_mask(json_path, img_shape, mask_path)
    else:
        print(f"❌ Image file not found for {json_file}")


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/Het/Desktop/Het/Method-2/images/IMP.json'

In [13]:
import os
import json
import cv2
import numpy as np

# Paths
image_dir = r"C:/Users/Het/Desktop/Het/Method-2/images"
json_dir = r"C:/Users/Het/Desktop/Het/Method-2/Json"
mask_dir = r"C:/Users/Het/Desktop/Het/Method-2/Mask"

# Ensure mask directory exists
os.makedirs(mask_dir, exist_ok=True)

# Process each JSON file
for json_file in os.listdir(json_dir):
    if not json_file.endswith(".json"):
        continue  # Skip non-JSON files

    json_path = os.path.join(json_dir, json_file)
    
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    img_metadata = data.get("_via_img_metadata", {})
    
    for img_id, img_info in img_metadata.items():
        filename = img_info.get("filename")
        regions = img_info.get("regions", [])

        if not filename:
            continue

        # Find the image
        img_path = None
        for folder in os.listdir(image_dir):
            folder_path = os.path.join(image_dir, folder)
            potential_path = os.path.join(folder_path, filename)
            if os.path.exists(potential_path):
                img_path = potential_path
                break
        
        if not img_path:
            print(f"Image not found for {filename}")
            continue

        # Read image dimensions
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read image: {img_path}")
            continue
        
        height, width = img.shape[:2]
        mask = np.zeros((height, width), dtype=np.uint8)

        # Draw rectangles
        for region in regions:
            shape = region.get("shape_attributes", {})
            if shape.get("name") == "rect":
                x, y, w, h = shape["x"], shape["y"], shape["width"], shape["height"]
                cv2.rectangle(mask, (x, y), (x + w, y + h), 255, thickness=-1)

        # Save the mask
        mask_filename = os.path.splitext(filename)[0] + "_mask.png"
        mask_path = os.path.join(mask_dir, mask_filename)
        cv2.imwrite(mask_path, mask)
        print(f"Saved mask: {mask_path}")


Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-02-05-a-p0001_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-02-05-a-p0002_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-02-05-a-p0003_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-02-05-a-p0004_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0001_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0002_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0003_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0004_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0005_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0006_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0007_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask\GDL-1798-05-29-a-p0008_mask.png
Save

In [14]:
import os
import json
import cv2
import numpy as np

# Paths
image_dir = r"C:/Users/Het/Desktop/Het/Method-2/images"
json_dir = r"C:/Users/Het/Desktop/Het/Method-2/Json"
mask_dir = r"C:/Users/Het/Desktop/Het/Method-2/Mask2"

# Ensure mask directory exists
os.makedirs(mask_dir, exist_ok=True)

# Process each JSON file
for json_file in os.listdir(json_dir):
    if not json_file.endswith(".json"):
        continue  # Skip non-JSON files

    json_path = os.path.join(json_dir, json_file)
    
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    img_metadata = data.get("_via_img_metadata", {})

    for img_id, img_info in img_metadata.items():
        filename = img_info.get("filename")
        regions = img_info.get("regions", [])

        if not filename:
            continue

        # Find the image
        img_path = None
        for folder in os.listdir(image_dir):
            folder_path = os.path.join(image_dir, folder)
            potential_path = os.path.join(folder_path, filename)
            if os.path.exists(potential_path):
                img_path = potential_path
                break
        
        if not img_path:
            print(f"Image not found for {filename}")
            continue

        # Read image dimensions
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read image: {img_path}")
            continue

        height, width = img.shape[:2]
        mask = np.zeros((height, width), dtype=np.uint8)

        if len(regions) == 0:
            print(f"Warning: No regions found for {filename}, skipping...")
            continue  # Skip this image since it has no annotations

        # Draw rectangles
        for region in regions:
            shape = region.get("shape_attributes", {})
            if shape.get("name") == "rect":
                x, y, w, h = shape.get("x", 0), shape.get("y", 0), shape.get("width", 0), shape.get("height", 0)

                # Ensure values are within valid bounds
                x, y = max(0, x), max(0, y)
                w, h = max(1, w), max(1, h)  # Ensure at least 1px size
                if x + w > width or y + h > height:
                    print(f"Skipping region: Bounding box exceeds image size for {filename}")
                    continue

                cv2.rectangle(mask, (x, y), (x + w, y + h), 255, thickness=-1)

        # Save the mask if it has annotations
        if np.any(mask > 0):  # Check if mask contains any white pixels
            mask_filename = os.path.splitext(filename)[0] + "_mask.png"
            mask_path = os.path.join(mask_dir, mask_filename)
            cv2.imwrite(mask_path, mask)
            print(f"Saved mask: {mask_path}")
        else:
            print(f"Skipping {filename}: Generated empty mask (black image)")


Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-04-28-a-p0001_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-04-28-a-p0002_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-04-28-a-p0003_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-05-26-a-p0001_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-05-26-a-p0002_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-08-04-a-p0001_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-08-04-a-p0002_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1837-08-04-a-p0003_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1842-03-18-a-p0001_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1842-03-18-a-p0002_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1842-03-18-a-p0003_mask.png
Saved mask: C:/Users/Het/Desktop/Het/Method-2/Mask2\GDL-1842-10-04-a-p0001_m

In [15]:
import os
path = r"C:/Users/Het/Desktop/Het/Method-2/top-10-english-newspapers-in-india1.png"
print("File exists:", os.path.exists(path))

File exists: False
